In [ ]:
# IMPORT AND DISPLAY SETTINGS


import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time
import mlflow
import mlflow.sklearn
import mlflow.lightgbm
import mlflow.xgboost

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, average_precision_score, ConfusionMatrixDisplay, roc_auc_score

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier



# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format



In [15]:
user_data = pd.read_csv(r"processed-data\loan-processed-data.csv")

In [16]:
# user_data.isna().sum().reset_index()
# No null values

In [17]:
user_data['Default'] = user_data['Default'].astype(int)
# Sanitize column names globally
user_data.columns = [col.strip().replace(" ", "_") for col in user_data.columns]

user_data.head()

,Client_Income_Type_Govt_Job,Client_Income_Type_Others,Client_Income_Type_Retired,Client_Income_Type_Service,Client_Income_Type_Unemployed,Client_Education_Graduation_dropout,Client_Education_Junior_secondary,Client_Education_Post_Grad,Client_Education_Secondary,Client_Marital_Status_M,Client_Marital_Status_S,Client_Marital_Status_W,Client_Gender_Male,Loan_Contract_Type_RL,Client_Housing_Type_Home,Client_Housing_Type_Municipal,Client_Housing_Type_Office,Client_Housing_Type_Rental,Client_Housing_Type_Shared,Client_Permanent_Match_Tag_Yes,Client_Contact_Work_Tag_Yes,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Accompany_Client,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Homephone_Tag,Workphone_Working,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Score_Source_2,Score_Source_3,Phone_Change,Credit_Bureau,Default,Income_to_Loan_Ratio,Installment_Rate,Per_Capita_Income,Has_Children,Is_Single_Parent,Employment_Percent_Life,Income_to_Region_Pop,Occupation_Risk_Level,Org_Type_Binned
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,6750.00,0.00,0.00,1.00,0.00,0.00,0.03,13957.00,1062.00,6123.00,383.00,1.00,0.00,2.00,6.00,17.00,0.48,0.55,63.00,1.00,0,0.11,0.06,3375.00,0.00,0.00,0.08,193.48,2.00,3.00
1,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,20250.00,1.00,0.00,1.00,1.00,0.00,0.01,14162.00,4129.00,7833.00,21.00,0.00,1.00,2.00,3.00,10.00,0.22,0.55,755.00,1.00,0,1.33,0.12,10125.00,0.00,0.00,0.29,173.64,2.00,2.00
2,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,18000.00,0.00,0.00,1.00,0.00,0.00,0.02,16790.00,5102.00,4493.00,331.00,0.00,0.00,2.00,4.00,12.00,0.55,0.33,277.00,0.00,0,0.30,0.05,9000.00,1.00,1.00,0.30,410.40,2.00,3.00
3,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,15750.00,0.00,0.00,1.00,1.00,0.00,0.01,23195.00,12019.50,4493.00,775.00,0.00,0.00,3.00,2.00,15.00,0.14,0.63,1700.00,3.00,0,0.29,0.04,7875.00,0.00,0.00,0.52,166.26,2.00,1.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,33750.00,1.00,0.00,1.00,0.00,0.00,0.02,11366.00,2977.00,5516.00,4043.00,0.00,0.00,1.00,3.00,12.00,0.30,0.36,674.00,1.00,0,0.25,0.03,8437.50,1.00,0.00,0.26,699.06,2.00,3.00


In [18]:
# user_data.isna().sum().reset_index()

In [20]:
# PEEK AT CO-RELATION HEATMAP
# corr = user_data.corr()
# plt.figure(figsize=(20, 10))  # Adjust size here as needed
# sns.heatmap(corr, annot=True, cmap="coolwarm")
# plt.title("Feature Correlation Heatmap")
# plt.tight_layout()

In [21]:

# Set MLflow experiment
mlflow.set_experiment("Loan Default Model Comparison V5")

# Columns to scale
columns_to_scale = [
    'Client_Income',
    'Population_Region_Relative', 'Age_Days', 'Employed_Days',
    'Registration_Days', 'ID_Days', 'Score_Source_2',
    'Score_Source_3', 'Phone_Change', 'Credit_Bureau'
]

# Split and scale
def stratified_split_and_scale(df, target_col='Default', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=test_size, random_state=random_state
    )

    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
    X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

    return X_train_scaled, X_test_scaled, X_train, X_test, y_train, y_test, scaler

# Resampling logic
def apply_resampling(X, y, method='none'):
    print(f"\n🔁 {method.upper()} - Before: {Counter(y)}")
    if method == 'undersample':
        sampler = RandomUnderSampler(random_state=42)
    elif method == 'oversample':
        sampler = RandomOverSampler(random_state=42)
    elif method == 'smote':
        sampler = SMOTE(random_state=42)
    elif method == 'smote_tomek':
        sampler = SMOTETomek(random_state=42)
    else:
        return X, y

    X_res, y_res = sampler.fit_resample(X, y)
    print(f"✅ {method.upper()} - After: {Counter(y_res)}")
    return X_res, y_res

# Evaluation logic
def evaluate_with_mlflow(model, X_test, y_test, model_name, sampling_method, X_train, y_train, train_time, custom_threshold):
    # custom_threshold = 0.3
    with mlflow.start_run(run_name=f"{model_name} with {sampling_method} with threshold {custom_threshold}" ):

        mlflow.set_tag("dataset", "loan_default_v2")
        # Log parameters
        mlflow.log_param("Model", model_name)
        mlflow.log_param("Sampling", sampling_method)
        mlflow.log_param("custom_threshold", custom_threshold)
        mlflow.log_param("Dataset", "loan_default_v2" )

        
# Check if model supports probability output
        pr_auc = None
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:, 1]
            y_pred = (y_proba >= custom_threshold).astype(int)

            # Log PR AUC
            pr_auc = average_precision_score(y_test, y_proba)
            mlflow.log_metric("PR_AUC", pr_auc)
        else:
            print(f"{model_name} does not support predict_proba. Skipping threshold logic and PR AUC.")
            y_pred = model.predict(X_test)
            y_proba = None  # Optional: can skip or leave undefined

        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        # pr_auc = average_precision_score(y_test, y_proba)

        # Log metrics
        mlflow.log_metric("Precision", report['1']['precision'])
        mlflow.log_metric("Recall", report['1']['recall'])
        mlflow.log_metric("F1", report['1']['f1-score'])
        mlflow.log_metric("Train_Time_seconds", round(train_time, 2))

        # Log model
        # mlflow.sklearn.log_model(model, model_name)
        
        if isinstance(model, xgb.XGBClassifier):
            mlflow.xgboost.log_model(model, name=model_name, input_example=X_test.iloc[:1])
        elif isinstance(model, lgb.LGBMClassifier):
            mlflow.lightgbm.log_model(model, name=model_name, input_example=X_test.iloc[:1])
        else:
            mlflow.sklearn.log_model(model, name=model_name, input_example=X_test.iloc[:1])

        # Confusion matrix
        fig, ax = plt.subplots()
        ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax)
        plt.title("Confusion Matrix")
        path = f"conf_matrix_{model_name}_{sampling_method}.png"
        plt.savefig(path)
        mlflow.log_artifact(path)
        plt.close()


        return {
            "Model": model_name,
            "Sampler": sampling_method,
            "Threshold": custom_threshold,
            "Precision": report['1']['precision'],
            "Recall": report['1']['recall'],
            "F1-Score": report['1']['f1-score'],
            "PR AUC": pr_auc,
            "Train Time (s)": round(train_time, 3)
    }



2025/06/23 16:09:28 INFO mlflow.tracking.fluent: Experiment with name 'Loan Default Model Comparison V5' does not exist. Creating a new experiment.


In [22]:

results = []

X_train_scaled, X_test_scaled, X_train_unscaled, X_test_unscaled, y_train, y_test, scaler = stratified_split_and_scale(
    user_data, target_col='Default'
)

resample_methods = ['none', 'undersample', 'oversample', 'smote', 'smote_tomek' ]
# resample_methods = ['none', 'undersample']

thresholds = [0.3, 0.5, 0.8]

for method in resample_methods:
    print(f"\n============================")
    print(f"🔄 Sampling: {method.upper()}")
    print("============================")

    # Logistic Regression
    X_res, y_res = apply_resampling(X_train_scaled, y_train, method)
    model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        result = evaluate_with_mlflow(model, X_test_scaled, y_test, "LogisticRegression", method, X_res, y_res, end - start, threshold)
        results.append(result)

    # evaluate_with_mlflow(model, X_test_scaled, y_test, "LogisticRegression", method, X_res, y_res, end - start)

    # Decision Tree
    X_res, y_res = apply_resampling(X_train_scaled, y_train, method)
    model = DecisionTreeClassifier(class_weight='balanced', max_depth=5, random_state=42)
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        result = evaluate_with_mlflow(model, X_test_scaled, y_test, "DecisionTree", method, X_res, y_res, end - start, threshold)
        results.append(result)

    # Random Forest
    X_res, y_res = apply_resampling(X_train_unscaled, y_train, method)
    model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        result = evaluate_with_mlflow(model, X_test_unscaled, y_test, "RandomForest", method, X_res, y_res, end - start, threshold)
        results.append(result)

    # XGBoost
    X_res, y_res = apply_resampling(X_train_unscaled, y_train, method)
    model = xgb.XGBClassifier(
        n_estimators=100,
        eval_metric='logloss',
        scale_pos_weight=(y_res == 0).sum() / (y_res == 1).sum(),  # handle imbalance
        random_state=42
    )
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        result = evaluate_with_mlflow(model, X_test_unscaled, y_test, "XGBoost", method, X_res, y_res, end - start, threshold)
        results.append(result)

    # LightGBM
    X_res, y_res = apply_resampling(X_train_unscaled, y_train, method)
    model = lgb.LGBMClassifier(
        n_estimators=100,
        class_weight='balanced',  # handles imbalance natively
        random_state=42
    )
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        result = evaluate_with_mlflow(model, X_test_unscaled, y_test, "LightGBM", method, X_res, y_res, end - start, threshold)
        results.append(result)

    # Balanced Random Forest (only on 'none')
    if method == 'none':
        model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
        start = time.time()
        model.fit(X_train_unscaled, y_train)
        end = time.time()
        for threshold in thresholds:
            result = evaluate_with_mlflow(model, X_test_unscaled, y_test, "BalancedRandomForest", method, X_train_unscaled, y_train, end - start, threshold)
            results.append(result)



🔄 Sampling: NONE

🔁 NONE - Before: Counter({0: 89608, 1: 7876})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that include


🔁 NONE - Before: Counter({0: 89608, 1: 7876})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co


🔁 NONE - Before: Counter({0: 89608, 1: 7876})


...

c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co


🔁 SMOTE_TOMEK - Before: Counter({0: 89608, 1: 7876})
✅ SMOTE_TOMEK - After: Counter({0: 88925, 1: 88925})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co


🔁 SMOTE_TOMEK - Before: Counter({0: 89608, 1: 7876})
✅ SMOTE_TOMEK - After: Counter({0: 88925, 1: 88925})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co

In [59]:
pd.set_option('display.max_rows', 100)

model_comparison_data = pd.DataFrame(results)
model_comparison_data.sort_values(by=['Recall','F1-Score','PR AUC'], ascending=False)

,Model,Sampler,Threshold,Precision,Recall,F1-Score,PR AUC,Train Time (s)
24,RandomForest,undersample,0.30,0.13,0.93,0.22,0.45,14.08
30,LightGBM,undersample,0.30,0.15,0.91,0.25,0.49,0.97
33,LogisticRegression,oversample,0.30,0.11,0.89,0.20,0.22,49.12
18,LogisticRegression,undersample,0.30,0.11,0.89,0.20,0.22,5.79
0,LogisticRegression,none,0.30,0.11,0.89,0.20,0.22,20.88
12,LightGBM,none,0.30,0.16,0.88,0.27,0.50,2.64
45,LightGBM,oversample,0.30,0.16,0.88,0.27,0.50,3.97
27,XGBoost,undersample,0.30,0.15,0.88,0.26,0.48,3.15
36,DecisionTree,oversample,0.30,0.14,0.87,0.24,0.43,4.45
3,DecisionTree,none,0.30,0.14,0.87,0.24,0.43,2.84


In [ ]:
# Good looking models based on visual analysis:
# RandomForest	undersample	0.50	0.22	0.75	0.34	0.45	14.08
# BalancedRandomForest	none	0.30	0.17	0.85	0.28	0.50	24.18
# XGBoost	undersample	0.50	0.21	0.75	0.33	0.48	3.15

In [68]:
pivot_df = model_comparison_data.pivot_table(
    index='Threshold',
    columns='Model',
    values=['Precision', 'Recall', 'F1-Score']
)
pivot_df

# Increasing threshold values is resulting in high recall but less precision if we average results for diffrent sampling techniques across different models

# Best threshold for our case(For High Recall) = 0.3 (or maybe 0.5 for balanced f1 score)

F1-Score                                           \
Model     BalancedRandomForest DecisionTree LightGBM LogisticRegression   
Threshold                                                                 
0.30                      0.28         0.25     0.33               0.20   
0.50                      0.41         0.31     0.37               0.27   
0.80                      0.34         0.24     0.39               0.25   

                                          Precision                        \
Model     RandomForest XGBoost BalancedRandomForest DecisionTree LightGBM   
Threshold                                                                   
0.30              0.44    0.35                 0.17         0.15     0.29   
0.50              0.39    0.39                 0.30         0.21     0.47   
0.80              0.16    0.40                 0.86         0.38     0.75   

                                                                Recall  \
Model     LogisticRegression RandomForest XGBoost BalancedRandomForest   
Threshold                                                                
0.30                    0.12         0.49    0.31                 0.85   
0.50                    0.17         0.80    0.47                 0.63   
0.80                    0.29         0.92    0.71                 0.21   

                                                                         
Model     DecisionTree LightGBM LogisticRegression RandomForest XGBoost  
Threshold                                                                
0.30              0.82     0.69               0.88         0.54    0.65  
0.50              0.60     0.53               0.68         0.35    0.51  
0.80              0.18     0.28               0.21         0.10    0.31

In [65]:
# Best performing param from each model
model_comparison_data.sort_values(by=['Model','Recall','F1-Score','PR AUC'], ascending=False).groupby('Model').first()


,Sampler,Threshold,Precision,Recall,F1-Score,PR AUC,Train Time (s)
Model,,,,,,,
BalancedRandomForest,none,0.30,0.17,0.85,0.28,0.50,24.18
DecisionTree,oversample,0.30,0.14,0.87,0.24,0.43,4.45
LightGBM,undersample,0.30,0.15,0.91,0.25,0.49,0.97
LogisticRegression,oversample,0.30,0.11,0.89,0.20,0.22,49.12
RandomForest,undersample,0.30,0.13,0.93,0.22,0.45,14.08
XGBoost,undersample,0.30,0.15,0.88,0.26,0.48,3.15


# Model Evaluation Report – Loan Default Prediction

## Objective

The goal is to identify the best-performing model for predicting loan defaults in the context of a highly imbalanced dataset. Evaluation is based on the following metrics:

- **Recall**: Priority metric to ensure most defaulters are correctly identified
- **F1-Score**: Balances precision and recall
- **PR AUC**: Area under the Precision-Recall curve, more informative than ROC AUC for imbalanced datasets

---

## Summary of Model Performance (Threshold = 0.30)

| Model                  | Sampler     | Precision | Recall | F1-Score | PR AUC |
|------------------------|-------------|-----------|--------|----------|--------|
| BalancedRandomForest   | none        | 0.17      | 0.85   | 0.28     | 0.50   |
| DecisionTree           | oversample  | 0.14      | 0.87   | 0.24     | 0.43   |
| LightGBM               | undersample | 0.15      | 0.91   | 0.25     | 0.49   |
| LogisticRegression     | oversample  | 0.11      | 0.89   | 0.20     | 0.22   |
| RandomForest           | undersample | 0.13      | 0.93   | 0.22     | 0.45   |
| XGBoost                | undersample | 0.15      | 0.88   | 0.26     | 0.48   |

---

## Analysis

- **Highest Recall**: RandomForest (0.93)
- **Highest F1-Score**: BalancedRandomForest (0.28)
- **Highest PR AUC**: BalancedRandomForest (0.50)
- **Most Balanced Trade-off**: BalancedRandomForest and XGBoost

---

## Recommendation

**Selected Model**: BalancedRandomForest

Justification:

- Achieves the highest F1-Score and PR AUC among all models
- Maintains a high recall (0.85), which is critical in identifying defaulters
- Offers a better balance between capturing defaulters and minimizing false positives compared to other models

**Alternative Option**: RandomForest

- Should be considered if the sole objective is to maximize recall
- Has the highest recall (0.93), but lower F1-Score (0.22), indicating a potential increase in false positives

---

## Next Steps

- Perform threshold tuning to further improve the balance between precision and recall
- Generate PR curves or threshold vs metric plots for deeper insight
- Evaluate the potential benefit of ensembling with models like XGBoost


In [ ]:
# import optuna
# import mlflow
# import mlflow.sklearn
# from imblearn.ensemble import BalancedRandomForestClassifier
# from sklearn.metrics import average_precision_score
# from sklearn.model_selection import StratifiedKFold, cross_val_predict
# from sklearn.pipeline import Pipeline
# import numpy as np
# from sklearn.metrics import (
#     average_precision_score,
#     recall_score,
#     f1_score,
#     precision_score,
#     accuracy_score
# )

# def stratified_split(df, target_col='Default', test_size=0.2, random_state=42):
#     X = df.drop(columns=[target_col])
#     y = df[target_col]

#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, stratify=y, test_size=test_size, random_state=random_state
#     )

#     return X_train, X_test, y_train, y_test

# def objective(trial):
#     # Define hyperparameter search space
#     params = {
#         'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300]),
#         'max_depth': trial.suggest_categorical('max_depth', [10, 20, None]),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
#         'sampling_strategy': trial.suggest_categorical('sampling_strategy', ['auto', 0.5, 0.8]),
#         'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
#         'replacement': trial.suggest_categorical('replacement', [False]),
#         'class_weight': trial.suggest_categorical('class_weight', ['balanced', None])
#     }

#     model = BalancedRandomForestClassifier(
#         **params,
#         random_state=42,
#         n_jobs=-1
#     )

#     # Cross-validation with prediction for metrics
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     y_pred_proba = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
#     score = average_precision_score(y_train, y_pred_proba)

#     # MLflow logging
#     with mlflow.start_run(nested=True):
#         mlflow.log_params(params)
#         mlflow.log_metric("average_precision", score)

#     return score


In [ ]:
# X_train, X_test, y_train, y_test = stratified_split(user_data)

# # Top-level MLflow experiment
# mlflow.set_experiment("Loan Default - BRF Optuna Tuning")

# # Launch Optuna study
# study = optuna.create_study(direction='maximize', study_name="BRF_Hyperparam_Tuning")
# with mlflow.start_run(run_name="Optuna_BRF_Tuning_Main"):
#     study.optimize(objective, n_trials=30, timeout=1200)

# # Log best model config
# mlflow.log_params(study.best_params)
# mlflow.log_metric("best_average_precision", study.best_value)
# print("Best Params:", study.best_params)


2025/06/23 19:42:57 INFO mlflow.tracking.fluent: Experiment with name 'Loan Default - BRF Optuna Tuning' does not exist. Creating a new experiment.
[I 2025-06-23 19:42:57,650] A new study created in memory with name: BRF_Hyperparam_Tuning
[I 2025-06-23 19:43:06,270] Trial 0 finished with value: 0.4252280624701917 and parameters: {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'log2', 'sampling_strategy': 0.8, 'bootstrap': False, 'replacement': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.4252280624701917.
[I 2025-06-23 19:43:13,231] Trial 1 finished with value: 0.4521827436016657 and parameters: {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'sampling_strategy': 'auto', 'bootstrap': True, 'replacement': False, 'class_weight': None}. Best is trial 1 with value: 0.4521827436016657.
[I 2025-06-23 19:43:30,186] Trial 2 finished with value: 0.48685852920

Best Params: {'n_estimators': 300, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'sampling_strategy': 0.5, 'bootstrap': False, 'replacement': False, 'class_weight': 'balanced'}


In [78]:
# ... [your imports]

def stratified_split(df, target_col='Default', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return train_test_split(X, y, stratify=y, test_size=test_size, random_state=random_state)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300]),
        'max_depth': trial.suggest_categorical('max_depth', [10, 20, None]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'sampling_strategy': trial.suggest_categorical('sampling_strategy', ['auto', 0.5, 0.8]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'replacement': trial.suggest_categorical('replacement', [False]),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None])
    }

    model = BalancedRandomForestClassifier(**params, random_state=42, n_jobs=-1)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_pred_proba = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Metrics
    recall = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    accuracy = accuracy_score(y_train, y_pred)
    pr_auc = average_precision_score(y_train, y_pred_proba)

    # Composite Score (normalized weighting)
    # composite_score = 0.6 * recall + 0.3 * f1 + 0.1 * pr_auc
    if precision < 0.25:
        composite_score = 0
    else:
        composite_score = 0.4 * recall + 0.3 * f1 + 0.2 * precision + 0.1 * pr_auc

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        mlflow.log_metrics({
            "recall": recall,
            "f1_score": f1,
            "precision": precision,
            "accuracy": accuracy,
            "pr_auc": pr_auc,
            "composite_score": composite_score
        })

    
    return composite_score

# Load your train/test data
X_train, X_test, y_train, y_test = stratified_split(user_data)

# Set MLflow experiment
mlflow.set_experiment("Loan Default - BRF Optuna Tuning V2")

# Safely end any pre-existing run
if mlflow.active_run():
    mlflow.end_run()

# Run Optuna optimization
study = optuna.create_study(direction='maximize', study_name="BRF_Hyperparam_Tuning")
with mlflow.start_run(run_name="Optuna_BRF_Tuning_Main"):
    study.optimize(objective, n_trials=30, timeout=1200)

    # Log best trial
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_composite_score", study.best_value)

print("Best Params:", study.best_params)


[I 2025-06-23 22:09:01,155] A new study created in memory with name: BRF_Hyperparam_Tuning
[I 2025-06-23 22:09:32,538] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 'log2', 'sampling_strategy': 'auto', 'bootstrap': False, 'replacement': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.0.
[I 2025-06-23 22:09:56,434] Trial 1 finished with value: 0.4663004861997409 and parameters: {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'log2', 'sampling_strategy': 0.8, 'bootstrap': False, 'replacement': False, 'class_weight': None}. Best is trial 1 with value: 0.4663004861997409.
[I 2025-06-23 22:10:29,445] Trial 2 finished with value: 0.4663893125712723 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': 'log2', 'sampling_strategy': 0.5, 'bootstrap': True, '

Best Params: {'n_estimators': 300, 'max_depth': None, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'sampling_strategy': 0.5, 'bootstrap': False, 'replacement': False, 'class_weight': None}


In [88]:
# Best params for our use-case
best_params_post_tuning = study.best_params
best_params_post_tuning

{'n_estimators': 300,
 'max_depth': None,
 'min_samples_split': 3,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'sampling_strategy': 0.5,
 'bootstrap': False,
 'replacement': False,
 'class_weight': None}

In [91]:

# Instantiate the model
model = BalancedRandomForestClassifier(**best_params_post_tuning, random_state=42, n_jobs=-1)

# Fit the model on training data
model.fit(X_train, y_train)


BalancedRandomForestClassifier(min_samples_leaf=2, min_samples_split=3,
                               n_estimators=300, n_jobs=-1, random_state=42,
                               replacement=False, sampling_strategy=0.5)

In [92]:

custom_threshold = 0.5

y_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= custom_threshold).astype(int)

# Log PR AUC
pr_auc = average_precision_score(y_test, y_proba)

# Predict on test data
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # Probability for class 1 (default)


print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

print("ROC AUC:", roc_auc_score(y_test, y_proba))
print("PR AUC :", average_precision_score(y_test, y_proba))

Confusion Matrix:
 [[20695  1708]
 [  867  1102]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     22403
           1       0.39      0.56      0.46      1969

    accuracy                           0.89     24372
   macro avg       0.68      0.74      0.70     24372
weighted avg       0.91      0.89      0.90     24372



NameError: name 'roc_auc_score' is not defined